In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import FloatImage
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import time
import numpy as np
import branca.colormap as cm

In [2]:
match_frame = pd.read_csv('../data/no_activity_latlon.csv')
match_frame.head()

,Unnamed: 0,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,Measure,Data_Value_Unit,...,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,n_lat,n_lon,UniqueID,o_lat,o_lon,distance
0,0,2017,TN,Tennessee,Knoxville,Census Tract,BRFSS,Unhealthy Behaviors,No leisure-time physical activity among adults...,%,...,LPA,4740000,4.709300e+10,Physical Inactivity,35.123371,-85.249368,4714000-47065010433,35.123371,-85.249368,0.0
1,1,2017,TN,Tennessee,Clarksville,Census Tract,BRFSS,Unhealthy Behaviors,No leisure-time physical activity among adults...,%,...,LPA,4715160,4.712510e+10,Physical Inactivity,36.019985,-83.940415,4740000-47093004100,36.019985,-83.940415,0.0
2,2,2017,TN,Tennessee,Chattanooga,Census Tract,BRFSS,Unhealthy Behaviors,No leisure-time physical activity among adults...,%,...,LPA,4714000,4.706501e+10,Physical Inactivity,35.055331,-85.333062,4714000-47065010901,35.055331,-85.333062,0.0
3,3,2017,TN,Tennessee,Chattanooga,Census Tract,BRFSS,Unhealthy Behaviors,No leisure-time physical activity among adults...,%,...,LPA,4714000,4.706500e+10,Physical Inactivity,36.003542,-83.939873,4740000-47093002900,36.003542,-83.939873,0.0
4,4,2017,TN,Tennessee,Knoxville,Census Tract,BRFSS,Unhealthy Behaviors,No leisure-time physical activity among adults...,%,...,LPA,4740000,4.709300e+10,Physical Inactivity,35.998590,-83.972815,4740000-47093003902,35.998590,-83.972815,0.0


In [3]:
# covert string to an float64
# have an error with string cannot be parsed , (df[pd.to_numeric(df.B, errors='coerce').isnull()]

#match_frame['PopulationCount'] = match_frame[pd.to_numeric(match_frame.PopulationCount,errors='coerce').isnull()]

In [4]:
geometry = [Point(xy) for xy in zip(match_frame.n_lon, match_frame.n_lat)]
# cluster_frame = cluster_frame.drop(['Latitude', 'Longitude'], axis=1)
match_geo = gpd.GeoDataFrame(match_frame, crs="EPSG:4326", geometry=geometry)

In [5]:
match_map = folium.Map(location = [35.123371, -85.249368], zoom_start = 12)
folium.TileLayer('Mapbox Control Room').add_to(match_map)

In [6]:
#pop = str(match_frame['PopulationCount'])

In [7]:
# create  4 bins and the label arg to return 1 of 4 colors depending on the value of population count
#match_frame['marker_color'] = pd.cut(match_frame['PopulationCount'], bins=4, 
                              #labels=['yellow', 'green', 'blue', 'red'])

In [8]:
for row_index, row_values in match_geo.iterrows():
    loc = [row_values['n_lat'], row_values['n_lon']]
    pop = 'Obese/No Activity Match:' + (' ') + 'Population Count: '+ row_values['PopulationCount']
    marker = folium.Marker(
        location = loc,
        icon = folium.Icon(color = 'red',icon="user-md", prefix = 'fa'),  #icon is not popping up ref:https://fontawesome.com/icons/weight?style=solid
#        radius=100,
        popup = pop,
        #color=row['marker_color'],
        fill=True,)
        #color=colormap(??????))  # I want to add color scale to population count (popup with higher pop count, the darker red)
                                    #how to add legend with the icons and color scale
        
    
    marker.add_to(match_map)

In [9]:
for row_index, row_values in match_geo.iterrows():
    loc = [row_values['o_lat'], row_values['o_lon']]
    marker = folium.Circle(
        location = loc,
        radius=200,
        fill=True,
        color='blue') 
    
    marker.add_to(match_map)

#use imgur to upload picture and get direct link of url and ref guide is: https://flashissue.zendesk.com/hc/en-us/articles/216491878-Host-an-image-imgur-com-
    url = ('https://i.imgur.com/erBuseO.jpg')
folium.plugins.FloatImage(url, bottom=75, left=80).add_to(match_map)

In [10]:
match_map

In [11]:
# save maps as html file 
match_map.save('../maps/obese_no_activity_match_map.html')  


In [12]:
cluster_match_map = folium.Map(location =   [36.1399427822, -86.7819906718], zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_match_map)

# inside the loop add each markwe to the cluster
for row_index, row_values in match_geo.iterrows():
    loc = [row_values['n_lat'], row_values['n_lon']]
    pop = 'Obese/No Activity Match:' + (' ') + 'Population Count: '+ row_values['PopulationCount']
    marker = folium.Marker(
        location = loc,
        icon = folium.Icon(color = 'red',icon="user-md", prefix = 'fa'),  #icon is not popping up ref:https://fontawesome.com/icons/weight?style=solid
#        radius=100,
        popup = pop,
        #color=row['marker_color'],
        fill=True,)
    
    marker.add_to(marker_cluster)

In [13]:
for row_index, row_values in match_geo.iterrows():
    loc = [row_values['o_lat'], row_values['o_lon']]
    marker = folium.Circle(
        location = loc,
        radius=200,
        fill=True,
        color='blue') 
    
    marker.add_to(marker_cluster)

#use imgur to upload picture and get direct link of url and ref guide is: https://flashissue.zendesk.com/hc/en-us/articles/216491878-Host-an-image-imgur-com-
    url = ('https://i.imgur.com/erBuseO.jpg')
folium.plugins.FloatImage(url, bottom=75, left=80).add_to(cluster_match_map)

In [14]:
cluster_match_map

In [15]:
# save map as html file 
cluster_match_map.save('../maps/obese_no_activity_cluster_match_map.html')  

In [16]:
# how to export html file from desktop to a url with http or https